In [1]:
import cherrypy
import json
import redis
import uuid
import requests
from redis.commands.json.path import Path
import time

In [2]:
publicid = "redis-15072.c77.eu-west-1-1.ec2.cloud.redislabs.com"
username = "default"
password = "53R8YAlL81zAHIEVcPjwjzcnVQoSPhzt"
redis_port = 15072

redis_client = redis.Redis(host = publicid, password = password, username = username, port = redis_port)
print("Is connected? ", redis_client.ping())

Is connected?  True


In [3]:
class Device(object):
    exposed = True
    
    def GET(self):
        # response = {'mac_address':[]}
        items = []
        keys = redis_client.keys("*:battery")
        print(len(keys))
        for key in keys:
            key = key.decode()
            items.append(key.split(":")[0])

        response = {"mac_address":items}
        
        return json.dumps(response)


class SpecificDevice:
    exposed = True

    def GET(self, *path, **query):
        mac_address = path[0]
        if mac_address == "":
            raise cherrypy.HTTPError(400, "Bad requested: missing MAC address")
        else:
            items = []
            keys = redis_client.keys("*:battery")
            for key in keys:
                key = key.decode()
                items.append(key.split(":")[0])
            if mac_address not in items:
                raise cherrypy.HTTPError(404, "Not found: invalid MAC address")

        if query['from'] == "":
            raise cherrypy.HTTPError(400, "Bad requested: missing start time")
        if query['to'] == "":
            raise cherrypy.HTTPError(400, "Bad requested: missing end time")
        battery_levels = redis_client.ts().range(f"{mac_address}:battery", int(query['from']), int(query['to']))
        power_plugs = redis_client.ts().range(f"{mac_address}:power", int(query['from']), int(query['to']))
        timestamps = [battery[0] for battery in battery_levels]
        battery = [battery[1] for battery in battery_levels]
        power = [power_plug[1] for power_plug in power_plugs]
        response = {
            "mac_address":mac_address,
            "timestamps":timestamps,
            "battery_levels":battery,
            "power_plugged":power
        }
        return json.dumps(response)

    def DELETE(self, *path, **query):
        mac_address = path[0]
        if mac_address == "":
            raise cherrypy.HTTPError(400, "Bad requested: missing MAC address")
        else:
            items = []
            keys = redis_client.keys("*:battery")
            for key in keys:
                key = key.decode()
                items.append(key.split(":")[0])
            if mac_address not in items:
                raise cherrypy.HTTPError(404, "Not found: invalid MAC address")

        redis_client.delete(f"{mac_address}:battery")
        redis_client.delete(f"{mac_address}:power")

In [52]:
if __name__=='__main__':
    conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher(), 'tools.sessions.on':True}}
    cherrypy.tree.mount(Device(), '/devices', conf)
    cherrypy.tree.mount(SpecificDevice(), '/device', conf)
    cherrypy.config.update({'server.socket_host': '0.0.0.0'})
    cherrypy.config.update({'server.socket_port': 8080})
    cherrypy.engine.start()
    cherrypy.engine.block()

[25/Jan/2023:09:16:52] ENGINE Bus STARTING
[25/Jan/2023:09:16:52] ENGINE Started monitor thread 'Autoreloader'.
[25/Jan/2023:09:16:52] ENGINE Serving on http://0.0.0.0:8080
[25/Jan/2023:09:16:52] ENGINE Bus STARTED


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f20d7ce-f06e-4d5b-8d58-338c733410f7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>